In [1]:
import numpy as np
import math
from linear_solvers import NumPyLinearSolver, HHL
from qiskit import Aer
from numpy.linalg import norm
from qiskit.quantum_info import Statevector

In [2]:
def make_nxn(matrix, vector):
    if matrix.shape[0] != matrix.shape[1]:
        # set n to the highest value between the rows and columns of matrix
        # by doing this, we can ensure that the matrix will be nxn and the vector will be nx1
        n = max(matrix.shape[0], matrix.shape[1])
        matrix = np.pad(matrix, ((0, n-matrix.shape[0]), (0, n-matrix.shape[1])), 'constant', constant_values = (0))
        vector.resize(1, n, refcheck=False)
        # create a matrix (lambda) that contains a small value to add to the matrix and vector
        # this will ensure that the matrix is invertible and still close in value to the original
        la = np.empty(n)
        la.fill(10**-6)
        la_mat = np.diag(la)
        matrix = matrix + la_mat
        vector = vector + la
    return matrix, vector

In [3]:
def make_2nx2n(matrix, vector):
    # this function assumes that the matrix is square
    # if matrix is not 2^n x 2^n, then set n to the next highest power of 2
    if not math.log(matrix.shape[0], 2).is_integer():
        n = 2**(math.ceil(math.log(matrix.shape[0], 2)))
        matrix = np.pad(matrix, ((0, n-matrix.shape[0]), (0, n-matrix.shape[1])), 'constant', constant_values = (0))
        vector.resize(1, n, refcheck=False)
        for i in range(n):
            if matrix[i][i] == 0:
                matrix[i][i] = 1
    return matrix, vector

In [4]:
def make_hermitian(matrix, vector):
    # this function assumes that the matrix is square
    # if the matrix is not hermitian, make it hermitian and alter the vector accordingly
    matrix_H = np.matrix(matrix).H
    if not np.array_equal(matrix, matrix_H):
        zeros = np.zeros((matrix.shape[0], matrix.shape[0]))
        matrix = np.block([[zeros, matrix_H],
                       [matrix, zeros]])
        vector_conj = np.conj(vector)
        vector = np.append(vector_conj, vector)
    return matrix, vector

In [5]:
def hhl_compatible(matrix, vector):
    mat = matrix.copy()
    vec = vector.copy()
    mat, vec = make_nxn(mat,vec)
    mat, vec = make_2nx2n(mat, vec)
    mat, vec = make_hermitian(mat, vec)
    return mat, vec

In [6]:
m= np.array([[19.58211765, -2.42352941,  9.69411765],
             [-2.42352941, 19.29411765,  4.58823529],
             [ 2.42352941, -4.82352941, 18.35294118]])
v = np.array([ 1.30305882, -0.98235294,  0.07058824])

In [7]:
m2, v2 = hhl_compatible(m, v)

In [8]:
print(m2)
print(v2)

[[ 0.          0.          0.          0.         19.58211765 -2.42352941
   2.42352941  0.        ]
 [ 0.          0.          0.          0.         -2.42352941 19.29411765
  -4.82352941  0.        ]
 [ 0.          0.          0.          0.          9.69411765  4.58823529
  18.35294118  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          1.        ]
 [19.58211765 -2.42352941  9.69411765  0.          0.          0.
   0.          0.        ]
 [-2.42352941 19.29411765  4.58823529  0.          0.          0.
   0.          0.        ]
 [ 2.42352941 -4.82352941 18.35294118  0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          1.          0.          0.
   0.          0.        ]]
[ 1.30305882 -0.98235294  0.07058824  0.          1.30305882 -0.98235294
  0.07058824  0.        ]


In [9]:
v2_norm = np.linalg.norm(v2)

In [10]:
backend = Aer.get_backend('aer_simulator')
hhl = HHL(1e-3, quantum_instance=backend)
quantum_solution = hhl.solve(m2, v2)

In [11]:
classical_solution = NumPyLinearSolver().solve(m2, v2/v2_norm)

In [12]:
quantum_solution.euclidean_norm

0.04941639906242655

In [13]:
classical_solution.euclidean_norm

0.04944199003885272

In [14]:
print(quantum_solution.state)

      ┌─────────────┐┌──────┐        ┌─────────┐
q4_0: ┤0            ├┤6     ├────────┤6        ├
      │             ││      │        │         │
q4_1: ┤1 circuit-97 ├┤7     ├────────┤7        ├
      │             ││      │        │         │
q4_2: ┤2            ├┤8     ├────────┤8        ├
      └─────────────┘│      │┌──────┐│         │
q5_0: ───────────────┤0     ├┤5     ├┤0        ├
                     │      ││      ││         │
q5_1: ───────────────┤1 QPE ├┤4     ├┤1 QPE_dg ├
                     │      ││      ││         │
q5_2: ───────────────┤2     ├┤3     ├┤2        ├
                     │      ││      ││         │
q5_3: ───────────────┤3     ├┤2 1/x ├┤3        ├
                     │      ││      ││         │
q5_4: ───────────────┤4     ├┤1     ├┤4        ├
                     │      ││      ││         │
q5_5: ───────────────┤5     ├┤0     ├┤5        ├
                     └──────┘│      │└─────────┘
  q6: ───────────────────────┤6     ├───────────
                    

In [15]:
sv = Statevector(quantum_solution.state)
sv = sv.data[512:520].real
norm = quantum_solution.euclidean_norm
solution = norm * sv / np.linalg.norm(sv)
(v2_norm * solution)[0:3]

array([ 0.06965146, -0.03806428, -0.01540482])

In [16]:
(v2_norm * classical_solution.state)[0:3]

array([ 0.06942201, -0.03852157, -0.01544537])